<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/p027_Preprocessing_Hard_Way.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install "tensorflow-text==2.8.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 17.0 MB/s 


In [5]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [6]:
import collections
import pathlib
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization
import tensorflow_datasets as tfds
import tensorflow_text as tf_text


référence : https://www.tensorflow.org/tutorials/load_data/text?hl=en


# Objectif
 * Créer un modèle capable de prédire l'un des 3 auteurs de l'iliade

# Récupération des données
 * Il y a ici beaucoup de preprocessing de texte à effectuer
   * `tf.data.TextLineDataset` pour convertir un /des fichiers en un tensorflow dataset
   * `url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'`
   * `filenames = ['cowper.txt', 'derby.txt', 'butler.txt']`
   * utiliser `utils.get_file(name, origin=DIRECTORY_URL + name)` pour télécharger les données brutes
   * La différence entre `tf.keras.utils.text_dataset_from_directory` et `tf.data.TextLineDataset` est que le premier traite chaque fichier comme une observation alors que le second traite chaque ligne d'un fichier comme une observation


# Tokenization
 * Création d'un tokenizer via `tf_text.UnicodeScriptTokenizer()`
 * Création d'une fonction qui lowerise et tokenize un batch
 * Utilisation de cette fonction sur les datasets via la méthode .map()
 * Parcourir le dataset en affichant quelques exemples
 * Compter la fréquence des tokens via un dico, comme ci-dessous :

```python
tokenized_ds = configure_dataset(tokenized_ds)
vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
```
 * Convertir les tokens en jetons via ` tf.lookup.StaticVocabularyTable()`, il faut mapper les tokens vers les entiers 2 à vocab_size+2 car le 0 est reservé au padding et le 1 au token oov, i.e. unknown. Voir ci-dessous :

```python
keys = vocab
values = range(2, len(vocab) + 2)  # Reserve `0` for padding, `1` for OOV tokens.

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)
```
 * Bon un peu bordélique, mais faudrait recréer une fonction qui prend un batch feature , label et qui va standardiser, tokeniser, vectoriser le texte

```python
def preprocess_text(text, label):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label
```

 * Tester cette fonction unitairement puis l'appliquer à tout le dataset via `.map()` methode

# Datasets stratégie
 * Créer un dataset de training et de test avec les méthodes `.skip()` et `.take()`

```python
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)
```


# Model, compile(), et fit()
 * Utilisez le même modèle que pour le notebook précédent

```python
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

model = create_model(vocab_size,3)

model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

history = model.fit(train_data, validation_data=validation_data, epochs=3)
```

# Créer modèle d'inférence
 * Il s'agit grosso modo de rajouter un layer de preprocessing
 * Et de rajouter un layer d'action sigmoid

```python
preprocess_layer = TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_text.case_fold_utf8,
    split=tokenizer.tokenize,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

preprocess_layer.set_vocabulary(vocab) # on connait deja le vocabulaire donc on utilise .set_() plutot que .adapt()

export_model = tf.keras.Sequential(
    [preprocess_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])
```

# Evaluer le modèle
Sur un dataset de raw strings on évalue notre modèle d'inférence

```python
# Create a test dataset of raw strings.
test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = configure_dataset(test_ds)

loss, accuracy = export_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))
```

# Tester le modèle

Jouer avec le modèle sur de nouvelles données. Créer un fake batch et regarder les prédictions

```python
inputs = [
    "Join'd to th' Ionians with their flowing robes,",  # Label: 1
    "the allies, and his armour flashed about him so that he seemed to all",  # Label: 2
    "And with loud clangor of his arms he fell.",  # Label: 0
]

predicted_scores = export_model.predict(inputs)
predicted_labels = tf.math.argmax(predicted_scores, axis=1)

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())
```

# Répéter l'exercice
Sur les données IMDB, répétez l'exercice, avec un simple TextVectorizer layer


```python
# Training set.
train_ds = tfds.load(
    'imdb_reviews',
    split='train[:80%]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True)

# Validation set.
val_ds = tfds.load(
    'imdb_reviews',
    split='train[80%:]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True)
```

On doit avoir 86% d'accuracy sur le test set avec le même modèle et trois époques